In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc 
import squidpy as sq
import glob
import gc
import gseapy as gp
from gseapy.parser import read_gmt
import seaborn as sns
import matplotlib.pyplot as plt
from pyscenic.aucell import aucell, derive_auc_threshold, create_rankings, GeneSignature
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from matplotlib_venn import venn2
from omnipath.interactions import import_intercell_network

In [ ]:
adata = sc.read_h5ad("new_data.h5ad")

### QC metrics demonstrate appropriate filtering of low-quality cells

In [ ]:
### Overall QC metrics of the spatial transcriptomics dataset

sns.set(style="whitegrid")
fig, axs = plt.subplots(1, 2, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
axs[0].set_ylabel("Cell count")
sns.histplot(
    adata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
axs[1].set_ylabel("Cell count")
sns.histplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)

)
plt.tight_layout()
plt.savefig("new_plots_copy/QC_metrics.png",bbox_inches="tight")
plt.close()


In [ ]:
### Comparison of QC metrics between obstructed and unobstructed tissues

fig, axs = plt.subplots(1, 2, figsize=(15, 4))

# (1) Total transcripts per cell
axs[0].set_title("Total transcripts per cell")
axs[0].set_ylabel("Cell count")
sns.histplot(
    data=adata.obs,
    x="total_counts",
    hue="obstructed",        
    kde=False,
    element="step",      
    stat="count",
    common_norm=False,
    ax=axs[0],
    alpha=0.5
)

# (2) Unique transcripts per cell
axs[1].set_title("Unique transcripts per cell")
axs[1].set_ylabel("Cell count")
sns.histplot(
    data=adata.obs,
    x="n_genes_by_counts",
    hue="obstructed",
    kde=False,
    element="step",
    stat="count",
    common_norm=False,
    ax=axs[1],
    alpha=0.5
)

plt.tight_layout()
plt.savefig("QC_metrics_by_group.png",bbox_inches="tight")
plt.close()

## Scoring framework (gene sets and methods, benchmarking)

### 1 Gene set selection for senescence scoring

In [ ]:
martin_sen_genes = ["CDKN1A", "TRIO", "NEAT1", "MAP3K20", "ITGB4", "CLDN4", "EZR", "CREB5", "FOSL2",
                    "CTNNA1", "FAM171A1", "EXOC4", "SMURF1", "ARAP2", "CACNA2D3",
                    "OSBPL3", "ST3GAL1", "AFAP1", "KLF4", "PLXNA1", "JUNB", "RHOB", "EMP2", "ATF3", 
                    "PDGFA", "SRD5A1", "SVIL", "BCR", "SPINT2", "RUNX1", "SH3D19", "SMAD2", "KIF13B",
                    "SRSF3", "MYH14", "HOXB9", "MYO9A", "NT5C2", "PDCD6IP", "ITGA3", "PARP8", 
                    "ARPC2", "CTSL", "ADSS", "TGIF1", "PLEKHA2", "ACTB", "ZMAT3", "GRIP1", "GPRC5A", 
                    "NFIB", "PICALM", "SYNE2", "PPP2R2A", "BTBD7", "LMNA", "YWHAQ", "DNAJC5"]
all_genesets = read_gmt("all_genesets.gmt")
all_genesets["martin_sen"] = martin_sen_genes

### 2 Senescence scoring methods

In [ ]:
# Select the epithelial cells
epithelial_cells = adata[adata.obs["broad_celltype"].isin(["Collecting duct", "Distal tubule", "PT", "TAL/LoH"])]

In [ ]:
#genesets lists
gene_sets = ['seneQuest_kidney', 'FRIDMAN_SENESCENCE_UP', 'KEGG_CELL_CYCLE', 
             'REACTOME_SASP', 'Reck_kidney_inflam', 'sen_mayo','martin_sen']

#### ssGSEA

In [ ]:
tissues = epithelial_cells.obs["source"].unique()
# loop for gene sets
for set_name, gene_lists in all_genesets.items():
    print(f"Running Gene Set: {set_name}")

    # loop for each tissue
    for tissue in tissues:
        print(f"Running ssGSEA for {tissue}")

        subset = epithelial_cells[epithelial_cells.obs["source"] == tissue].copy()
        sub_geneset = {set_name: gene_lists}
        exp_df=subset.to_df().T

        ss = gp.ssgsea(data=exp_df,
               gene_sets=sub_geneset,
               outdir=None,
               sample_norm_method='rank',
               no_plot=True)

        ss_df = ss.res2d.pivot(index="Name", columns="Term", values="NES")
        ss_df.index.name = "cell_id"
        ss_df["source"] = tissue
        ss_df["gene_set"] = set_name
       
        ss_df.to_csv(f"new_copy/new_ssgsea_{set_name}_{tissue}.csv")
        
        del subset, ss, ss_df, sub_geneset
        gc.collect()

In [ ]:
# Make sure the index is string format
epithelial_cells.obs.index = epithelial_cells.obs.index.astype(str)

for gene_set in gene_sets:
    print(f"Processing {gene_set}")
    score_col_name = f"ssGSEA_{gene_set}_scores"

    # Search for the result files of all tissues
    file_list = glob.glob(f"new_copy/new_ssgsea_{gene_set}_*.csv")

    df_list = []
    for file in file_list:
        df = pd.read_csv(file)
        df = df[["cell_id", gene_set]]
        df.rename(columns={gene_set: score_col_name}, inplace=True)
        df.set_index("cell_id", inplace=True)
        df.index = df.index.astype(str)
        df_list.append(df)

    # Merge
    merged_df = pd.concat(df_list)
    merged_df = merged_df[~merged_df.index.duplicated(keep="first")]

    # Merge into epithelial_cells.obs
    epithelial_cells.obs = epithelial_cells.obs.join(merged_df, how="left")

#### AUCell

In [ ]:
tissues = epithelial_cells.obs["source"].unique()

# loop for gene sets
for set_name, gene_list in all_genesets.items():
    # create GeneSignature object
    gene2weight = {gene: 1.0 for gene in gene_list if gene in epithelial_cells.var_names}
    signature = GeneSignature(set_name, gene2weight)

    # loop for each tissue
    for tissue in tissues:
        print(f"\n Running AUCell:Gene Set={set_name}, Tissue={tissue}")

        # Extract the subset
        subset = epithelial_cells[epithelial_cells.obs["source"] == tissue].copy()
        exp_array = subset.layers["raw_counts"]
        exp_df = pd.DataFrame.sparse.from_spmatrix(exp_array, 
                              columns=subset.var_names, 
                              index=subset.obs_names)
        # Create a ranking object
        rankings = create_rankings(exp_df)

        # the AUCell score
        auc_mtx = aucell(rankings, [signature])
        auc_mtx.index.name = "cell_id"
        auc_mtx["source"] = tissue
        auc_mtx["gene_set"] = set_name

        # save
        auc_mtx.to_csv(f"new_copy/aucell_scores_{set_name}_{tissue}.csv")

        # memory Clear
        del subset, exp_array, exp_df, rankings, auc_mtx
        gc.collect()

In [ ]:
# Make sure the index is string format
epithelial_cells.obs.index = epithelial_cells.obs.index.astype(str)

for gene_set in gene_sets:
    print(f"Processing {gene_set}")
    score_col_name = f"AUCell_{gene_set}_scores"

    # Search for the result files
    files = glob.glob(f"new_copy/aucell_scores_{gene_set}_*.csv")

    df_list = []
    for file in files:
        df = pd.read_csv(file)
        df = df[["cell_id", gene_set]]
        df.rename(columns={gene_set: score_col_name}, inplace=True)
        df.set_index("cell_id", inplace=True)
        df.index = df.index.astype(str)
        df_list.append(df)

    # merge
    merged_df = pd.concat(df_list)
    merged_df = merged_df[~merged_df.index.duplicated(keep="first")]

    # Merge into .obs
    epithelial_cells.obs = epithelial_cells.obs.join(merged_df, how="left")

#### Scanpy

In [ ]:
# loop for each gene set
for set_name, gene_list in all_genesets.items():
    # Filter out vaild genes
    valid_genes = [g for g in gene_list if g in epithelial_cells.var_names]

    score_column = f"Scanpy_{set_name}_scores"

    sc.tl.score_genes(
        epithelial_cells,
        gene_list=valid_genes,
        score_name=score_column,
        use_raw=False)
    
    print(f"{score_column} added ({len(valid_genes)} genes)")

#### Martin's

In [ ]:
# expression matrix (cells × genes)
df_all = epithelial_cells.to_df()

# loop for each gene set
for set_name, gene_list in all_genesets.items():
    # Filter out vaild genes
    valid_genes = [g for g in gene_list if g in epithelial_cells.var_names]

    # scale the expression
    df_subset = df_all[valid_genes].copy()
    scaler = preprocessing.MinMaxScaler()
    df_subset[valid_genes] = scaler.fit_transform(df_subset)

    # sum as the score
    epithelial_cells.obs[f"Martin_{set_name}_scores"] = df_subset.sum(axis=1)

    print(f"{set_name} score added: {len(valid_genes)} genes")

### Violin plot and histogram for each scpring method in each tissue

In [ ]:
scoring_methods = ["ssGSEA","AUCell","Martin","Scanpy"]

In [ ]:
for method in scoring_methods:
    for gene_set in all_genesets:

        score_col = f"{method}_{gene_set}_scores"

        plt.figure(figsize=(12, 5))

        # Violin plot by tissue
        plt.subplot(1, 2, 1)
        sns.violinplot(
            data=epithelial_cells.obs,
            x="source",
            y=score_col,
            hue="obstructed",
            cut=0)
        plt.title(f"Violin plot of {score_col}")
        plt.xticks(rotation=45)
        plt.legend(title="Obstruction", loc="upper right")

        # Histogram (overall score distribution)
        plt.subplot(1, 2, 2)
        sns.histplot(
            data=epithelial_cells.obs,
            x=score_col,
            hue="obstructed",
            kde=True,
            element="step",
            stat="density", 
            common_norm=False )
        plt.title(f"Histogram of {score_col}")
        plt.xlabel(score_col)

        plt.tight_layout()
        plt.savefig(f"new_plots_copy/{score_col}.png")
        plt.close()

#### statistical test

In [ ]:
score_cols = [
    'ssGSEA_FRIDMAN_SENESCENCE_UP_scores',
    'ssGSEA_REACTOME_SASP_scores',
    'ssGSEA_sen_mayo_scores']

In [ ]:
# permutation test
res_list = []

for sc in score_cols:
    #take the median as the representative value
     agg = (
        df.groupby(tissue_col)[sc]
          .median()
          .rename('score_median')
          .reset_index()
        .merge(tissue_group[[tissue_col, 'Group_at_tissue']], on=tissue_col, how='left')
    )
    # grouping
    labels = agg['Group_at_tissue'].astype(str).str.lower()
    # obstructed
    g1 = agg.loc[labels.eq("obstructed"), 'score_median'].to_numpy() 
    # unobstructed
    g0 = agg.loc[~labels.eq("obstructed"), 'score_median'].to_numpy()
   
    # mean difference between groups
    obs_diff = g1.mean() - g0.mean()
    mwu_p = stats.mannwhitneyu(g1, g0, alternative='two-sided').pvalue


    #permutation test
    x = agg['score_median'].to_numpy()
    n1 = g1.size
    n = x.size
    rng = np.random.default_rng(0)

    B = 100000
    cnt = 0
    for _ in range(B):
        idx = rng.permutation(n)
        diff = x[idx[:n1]].mean() - x[idx[n1:]].mean()
        if abs(diff) >= abs(obs_diff):
                cnt += 1
            p_perm = (cnt + 1) / (B + 1)

    res_list.append({
        "score": sc,
        "mean_diff_ob_minus_un":float(obs_diff),
        "MWU_p":float(mwu_p),
        "perm_p": float(p_perm),
    })

pd.DataFrame(res_list)

In [ ]:
# OLS with tissue-clustered standard errors
rows=[]
for sc in score_cols:
    mod = smf.ols(f"{sc} ~ C({group_col})", data=df).fit(
        cov_type="cluster", 
        cov_kwds={"groups": df[tissue_col], "use_correction": True}
    )
    
    name = [i for i in mod.params.index if i.startswith(f"C({group_col})")][0]
    rows.append({
        "score": sc,
        "beta": mod.params[name], "se": mod.bse[name], "p": mod.pvalues[name],
        "ci_low": mod.conf_int().loc[name,0], "ci_high": mod.conf_int().loc[name,1],
        "method": "OLS + cluster-robust by tissue"
    })

out = pd.DataFrame(rows)
out["p_adj_fdr"] = multipletests(out["p"], method="fdr_bh")[1]

print(out)


### Thresholding and classification of senescent cells

In [ ]:
# choosing by manual
score = {
    "SASP": "ssGSEA_REACTOME_SASP_scores",
    "Mayo": "ssGSEA_sen_mayo_scores",
    "Friman": "ssGSEA_FRIDMAN_SENESCENCE_UP_scores"}

In [ ]:
# loop for all method
for label, score_col in score.items():
    
    # grouping extraction
    unobstructed = epithelial_cells.obs[epithelial_cells.obs["obstructed"] == "Unobstructed"]
    obstructed = epithelial_cells.obs[epithelial_cells.obs["obstructed"] == "Obstructed"]
    
    # cutoff：unobstructed -98% 
    cutoff = unobstructed[score_col].quantile(0.98)
    
    # Label obstructed cells: True - senescent
    senescent_label = (epithelial_cells.obs[score_col] > cutoff)
    epithelial_cells.obs[f"senescent_{label}"] = senescent_label
    
    # plot + cutoff
    plt.figure(figsize=(7, 4))
    sns.histplot(
        unobstructed[score_col], bins=50, kde=True, 
        label="Unobstructed", 
        color="orange" )
    sns.histplot(
        obstructed[score_col], bins=50, kde=True, 
        label="Obstructed", 
        color="blue")
    
    # cutoff line
    plt.axvline(cutoff, color='black', linestyle='--')
    plt.text(cutoff, plt.ylim()[1]*0.9, f"Cutoff = {cutoff:.2f}", ha='left')
    plt.xlabel("Senescence Score")
    plt.ylabel("Cell Count")
    plt.title(f"Threshold-Based Classification of Senescent Cells")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"new_plots_copy/Threshold/{score_col}_Threshold.png")
    plt.close()

    #  Welch’s t-test
    stat, p = ttest_ind(
    obstructed[score_col].dropna(), 
    unobstructed[score_col].dropna(), 
    equal_var=False)
    print(f"T-test (Welch): t = {stat:.2f}, p-value = {p:.3e}")

    stat, p = mannwhitneyu(
    obstructed[score_col].dropna(), 
    unobstructed[score_col].dropna(), 
    alternative="two-sided" )
    print(f"Mann-Whitney U test: U = {stat:.2f}, p = {p:.3e}")

#### Composite Senescence Score

In [ ]:
# Standardisation
fridman_z = zscore(epithelial_cells.obs["ssGSEA_FRIDMAN_SENESCENCE_UP_scores"])
mayo_z = zscore(epithelial_cells.obs["ssGSEA_sen_mayo_scores"])

# averaging Z-scores
epithelial_cells.obs["senescence_combined_scores"] = (fridman_z + mayo_z) / 2

In [ ]:
# calculate cutoff（Unobstructed- top 2%）
cutoff = epithelial_cells.obs.loc[
    epithelial_cells.obs["obstructed"] == "Unobstructed",
    "senescence_combined_scores"].quantile(0.98)

epithelial_cells.obs["senescent_final_scores_all"] = (
    epithelial_cells.obs["senescence_combined_scores"] > cutoff
)

plt.figure(figsize=(7, 4))
sns.histplot(
    epithelial_cells.obs.loc[epithelial_cells.obs["obstructed"] == "Unobstructed","senescence_combined_scores"],
    bins=50, 
    kde=True, 
    label="Unobstructed", 
    alpha=0.6
)
sns.histplot(
    epithelial_cells.obs.loc[epithelial_cells.obs["obstructed"] == "Obstructed","senescence_combined_scores"],
    bins=50, kde=True, label="Obstructed",
    alpha=0.6
)
plt.axvline(cutoff, color='black', linestyle='--', label=f"Cutoff = {cutoff:.2f}")
plt.title("Combined Senescence Scores Distribution (Fridman + Mayo)")
plt.xlabel("Composite Z-score")
plt.ylabel("Cell Count")
plt.legend()
plt.tight_layout()
plt.savefig(f"new_plots/senescent_final_scores.png", dpi=300)
plt.close()

# 不用筛选 senescent_final_scores
scores_ob = epithelial_cells.obs.loc[epithelial_cells.obs["obstructed"] == "Obstructed", "senescence_combined_scores"]
scores_un = epithelial_cells.obs.loc[epithelial_cells.obs["obstructed"] == "Unobstructed", "senescence_combined_scores"]


### Validation of classification comparing with p21 expression

In [ ]:
# he positive threshold of p21
epithelial_cells.obs["p21_positive"] = epithelial_cells.to_df()["CDKN1A"] > 0

summary = (
    epithelial_cells.obs
    .groupby("source")
    .apply(lambda df: pd.Series({
        "Senescent cells": df["senescent_final_scores_all"].sum(),
        "p21+ cells": df["p21_positive"].sum(),
        "Overlap": ((df["senescent_final_scores_all"]) & (df["p21_positive"])).sum()
    }))
    .reset_index()
)

# proportion
summary["Total cells"] = epithelial_cells.obs.groupby("source").size().values
summary["% p21+"] = (summary["p21+ cells"] / summary["Total cells"] * 100).round(2)
summary["% Senescent"] = (summary["Senescent cells"] / summary["Total cells"] * 100).round(2)

# sort by tissue
summary = summary.sort_values("source")

summary.to_csv("new_copy/senescent_vs_p21_by_tissue.csv", index=False)

In [ ]:
### Venn
p21_positive = set(epithelial_cells[epithelial_cells.obs["p21_positive"]].obs_names)
score_positive = set(epithelial_cells[epithelial_cells.obs["senescent_final_scores_all"]].obs_names)

venn2([set(p21_positive), set(score_positive)],
      set_labels=("p21-positive", "Composite score-positive"))

plt.savefig("new_plots_copy/veen.png")

###  Morphological PCA

In [ ]:
#Select the variables for PCA
morpho_features = [
    'area_x',
    'perimeter',
    'eccentricity',
    'circularity',
    'convexity',
    'aspect_ratio',
    'extent',
    'solidity',]

In [ ]:
# preparing 
df_pca = epithelial_cells.obs[morpho_features + ["senescent_final_scores_all"]].dropna()
X = df_pca[morpho_features].values
y = df_pca[sen_col].values

# normalisation
X_scaled = StandardScaler().fit_transform(X)

# pca
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_pca["PC1"] = X_pca[:, 0]
df_pca["PC2"] = X_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=df_pca,
    x="PC1", y="PC2",
    hue=sen_col,
    alpha=0.6,
    s=10)

plt.title(f"PCA of Morphological Features")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.legend(title="Senescence", loc="upper right")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"new_plots_copy/PCA.png")
plt.close()


In [ ]:
## final_PCA: Top Contributors to PC2
# Feature list
features = morpho_features

# loadings
pc2_loadings = pca.components_[1]

# sort the features and contribution coefficients
pc2_contrib = pd.Series(pc2_loadings, index=features)
pc2_contrib = pc2_contrib.sort_values(key=abs, ascending=False)

print("Top contributors to PC2:")
print(pc2_contrib.head(10))

## Merged sort

In [ ]:
# write the new column in epithelial_cells.obs to adata.obs
cols_to_merge = ['senescent_final_scores']

for col in cols_to_merge:
    adata.obs[col] = None  
    adata.obs.loc[epithelial_cells.obs.index, col] = epithelial_cells.obs[col]

In [ ]:
# create new cell type
sen = adata.obs["senescent_final_scores"].fillna(False)
new_Celltype = adata.obs["broad_celltype"].copy()

if pd.api.types.is_categorical_dtype(new_Celltype):
    new_Celltype = new_Celltype.cat.add_categories(["epithelial", "epithelial_sen"])

epi = adata.obs["broad_celltype"].isin(["Collecting duct", "Distal tubule", "PT", "TAL/LoH"])
new_Celltype.loc[epi & sen] = "epithelial_sen"
new_Celltype.loc[epi & ~sen] = "epithelial"

adata.obs["Celltype_sen"] = new_Celltype

In [ ]:
# remove old cell types
to_remove = ["Collecting duct", "Distal tubule", "PT", "TAL/LoH"]
adata.obs["Celltype_sen"] = adata.obs["Celltype_sen"].cat.remove_categories(to_remove)

## spatial neighbours

In [ ]:
for tissue in adata.obs["source"].unique():
    print(f"Processing: {tissue}")
    
    adata_sub = adata[adata.obs["source"] == tissue].copy()
    small_cats = adata_sub.obs["Celltype_sen_all"].value_counts()[lambda x: x < 10].index
    # drop small cats
    adata_sub = adata_sub[~adata_sub.obs["Celltype_sen_all"].isin(small_cats)].copy()
    
    sq.gr.spatial_neighbors(adata_sub, coord_type="generic")
    
    sq.gr.nhood_enrichment(adata_sub, cluster_key="Celltype_sen_all")

    sq.pl.nhood_enrichment(adata_sub,
                           cluster_key="Celltype_sen_all",
                           method= None,
                           cmap="coolwarm")
    plt.title(f"Neighbourhood Enrichment - {tissue}")
    plt.savefig(f"new_plots_copy/nhood_enrichment_{tissue}_copy.png",bbox_inches='tight')
    plt.close()

    enrichment = adata_sub.uns["Celltype_sen_nhood_enrichment"]["zscore"]
    celltypes = adata_sub.obs["Celltype_sen"].cat.categories
    enrichment_df = pd.DataFrame(enrichment, index=celltypes, columns=celltypes)
    enrichment_df.to_csv(f"new_copy/nhood_enrichment_{tissue}.csv")
    

In [ ]:
for tissue in adata.obs["source"].unique():
    print(f"Processing: {tissue}")

    adata_sub = adata[adata.obs["source"] == tissue].copy()
    
    # check small_cats
    small_cats = adata_sub.obs["Celltype_sen"].value_counts()[lambda x: x < 10].index
    # drop small cats
    adata_sub = adata_sub[~adata_sub.obs["Celltype_sen"].isin(small_cats)].copy()
    adata_sub.obs["Celltype_sen"] = adata_sub.obs["Celltype_sen"].astype(str)
   
    # calculate numer of senescent cells
    num_sen = len(adata_sub[adata_sub.obs["Celltype_sen"] == "epithelial_sen"])
    # get rid of epithelial_sen subtype
    adata_sub.obs["Celltype_sen"] = adata_sub.obs["Celltype_sen"].replace({"epithelial_sen": "epithelial" })
    #look only at epithelial cells
    ep_subset = adata_sub[adata_sub.obs["Celltype_sen"] == "epithelial"]
    #take random sample and make them "test" cells
    test_index = ep_subset.obs.sample(n= num_sen, random_state = 42).index
    
    adata_sub.obs.loc[test_index, "Celltype_sen"] = "epithelial_test"
    adata_sub.obs["Celltype_sen"] = adata_sub.obs["Celltype_sen"].astype("category")
    
    sq.gr.spatial_neighbors(adata_sub, coord_type="generic")

    sq.gr.nhood_enrichment(adata_sub, cluster_key="Celltype_sen")

    sq.pl.nhood_enrichment(adata_sub,
                           cluster_key="Celltype_sen",
                           method=None,
                           cmap="coolwarm")
    plt.title(f"Neighbourhood Enrichment - {tissue}")
    plt.savefig(f"new_plots_copy/nhood_enrichment_{tissue}_random_copy.png",bbox_inches='tight')
    plt.close()
    
    enrichment = adata_sub.uns["Celltype_sen_nhood_enrichment"]["zscore"]
    celltypes = adata_sub.obs["Celltype_sen"].cat.categories
    enrichment_df = pd.DataFrame(enrichment, index=celltypes, columns=celltypes)
    enrichment_df.to_csv(f"new_copy/nhood_enrichment_{tissue}.csv")

## Ligand-receptor analysis 

In [ ]:
# information about ligand-receptors
lr = import_intercell_network(
    transmitter_params={"categories": "ligand"},
    receiver_params={"categories": "receptor"} )

# Extract gene names
ligands = set(lr["source"])
receptors = set(lr["target"])

In [ ]:
# Mark to adata
adata.var["categories"] = None
adata.var.loc[adata.var_names.isin(ligands), "categories"] = "ligand"
adata.var.loc[adata.var_names.isin(receptors), "categories"] = adata.var["categories"].fillna("receptor")

In [ ]:
res = sq.gr.ligrec(
    adata,
    n_perms=1000,
    cluster_key="Celltype_sen",
    copy=True,
    use_raw=False,
    corr_method="fdr_bh",
    transmitter_params={"categories": "ligand"},
    receiver_params={"categories": "receptor"},
)

#Save
res["means"].to_csv("new_copy/ligrec_means.csv")
res["pvalues"].to_csv("new_copy/ligrec_pvalues.csv")
res["metadata"].to_csv("new_copy/ligrec_metadata.csv", index=False)

#### All Paris

In [ ]:
expr = res["means"]["epithelial_sen"]
pval = res["pvalues"]["epithelial_sen"]

# Combined conditions
combined_mask = (expr > 1.0) & (pval < 0.05)

#  filtered pairing index
selected_pairs = expr[combined_mask].index.tolist()

res_sub = {
    "means": means.loc[selected_pairs],
    "pvalues": pvalues.loc[selected_pairs],
}

sq.pl.ligrec(res_sub, source_groups="epithelial_sen")
plt.savefig(f"new_plots_copy/All_Receptor-ligand_sen.png",bbox_inches='tight')
plt.close()

sq.pl.ligrec(res_sub, source_groups="epithelial")
plt.savefig(f"new_plots_copy/All_Receptor-ligand_non.png",bbox_inches='tight')
plt.close()

#### Max

In [ ]:
# conditions of both max and mean 
top_expr = expr[(expr.max(axis=1) > 1) & (expr.mean(axis=1) > 0)]

top_pairs = top_expr.head(80).index.tolist()

# result dict
res_top = {
    "means":res["means"].loc[top_pairs],
    "pvalues": res["pvalues"].loc[top_pairs],
}

sq.pl.ligrec(res_top, source_groups="epithelial_sen")
plt.savefig(f"new_plots_copy/Max_Receptor-ligand_sen_top80.png")
plt.close()

sq.pl.ligrec(res_top, source_groups="epithelial")
plt.savefig(f"new_plots_copy/Max_Receptor-ligand_non_top80.png")
plt.close()